# Kernel Ridge Regression
This notebook takes the full data scaled to zero mean and unit variance and the PCA reduced data and trains a Kernel Ridge Regressor for both datasets.

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))

import numpy as np
import matplotlib.pyplot as plt
from src.kernels.rbf import RBFKernel
from src.models.kernel_ridge import KernelRidge

In [2]:
pca_data = np.load("../data/X_pca.npy")
scaled_data = np.load("../data/X_scaled.npy")
Y = np.load("../data/y.npy")

In [3]:
train_size = 8000

pca_train = pca_data[:train_size, :]
pca_val = pca_data[train_size:, :]
scaled_train = scaled_data[:train_size, :]
scaled_val = scaled_data[train_size:, :]
y_train = Y[:train_size]
y_val = Y[train_size:]

### Create RBF kernel function
Utillizes the fact that $(x-x')^2=||x||^2+||x'||^2-2x^Tx'$, 
https://www.youtube.com/watch?v=nJz_lazAUqw

In [4]:
kernel = RBFKernel(gamma=1.0)
model = KernelRidge(kernel=kernel, lambd=1e-2)
model.fit(scaled_train, y_train)
predictions = model.predict(scaled_val)
print("The absolute mean error is:", np.sum(np.abs(y_val - predictions))/ len(y_val))

The absolute mean error is: 12.217982961898688


In [5]:
best_gamma = None
best_lambda = None
best_MAE = np.inf
gamma_values = [0.01, 0.1, 1, 10]
lambda_values = [1e-3, 1e-2, 1e-1, 1]
for gamma in gamma_values:
    kernel = RBFKernel(gamma)
    for lambd in lambda_values:
        model = KernelRidge(kernel, lambd)
        model.fit(scaled_train, y_train)
        predictions = model.predict(scaled_val)
        mean_error = np.sum(np.abs(y_val - predictions))/ len(y_val)
        if mean_error < best_MAE:
            best_gamma = gamma
            best_lambda = lambd
            best_MAE = mean_error
print("The absolute mean error is:", best_MAE)

The absolute mean error is: 9.857950057617773
